In [115]:
import numpy as np
import matplotlib.pyplot as plt

In [116]:
def replace_outliers_iqr(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    original_data = data.copy()

    # Replace outliers with the nearest bound
    data[data < lower_bound] = lower_bound
    data[data > upper_bound] = upper_bound
    
    # plt.figure(figsize=(5, 5))
    # plt.plot(original_data, label='Original Data', color='blue', marker='o', linestyle='--')
    # plt.plot(data, label='Filtered Data', color='red', marker='o', linestyle='--')
    # plt.title('Original vs Filtered Data')
    # plt.xlabel('Index')
    # plt.ylabel('Value')
    # plt.legend()
    # plt.grid(True)
    # plt.show()

    return data

# Example usage:
# time_series = np.array([1, 2, 3, 10, 5, 6, 7, 8, 9, 10])
# filtered_series = remove_outliers_iqr(time_series)
# print("Original time series:", time_series)
# print("Filtered time series without outliers:", filtered_series)


In [117]:
# def remove_outlier(time_points):

In [118]:
# Load tapping data from a file with moving average
def load_tapping_data(file_path):
    tapping_times = None
    with open(file_path, "r") as file:
        tapping_times = file.read()
    time_points = []
    for line in tapping_times.split('\n'):
        if line.strip():
            time_points.append(float(line.split()[0]))

    # Divide timestamps into three lists based on their duration
    synchr_0_15 = []
    cont_music_15_45 = []
    cont_no_music_45_75 = []

    for data in time_points:
        if data < 15:
            synchr_0_15.append(data)
        elif data < 45:
            cont_music_15_45.append(data)
        else:
            cont_no_music_45_75.append(data)

    tapping_intervals = [np.diff(synchr_0_15),np.diff(cont_music_15_45),np.diff(cont_no_music_45_75)]

    for i in range(len(tapping_intervals)):
        tapping_intervals[i] = replace_outliers_iqr(tapping_intervals[i])

    return tapping_intervals

In [119]:
song_ids = ['A','B','C']
bpms = {'A':122, 'B':51, 'C':109}
file_paths = [f'./timestamp_files/timestamps_Mihika_{song_id}.txt' for song_id in song_ids]

for song_id, file_path in zip(song_ids, file_paths):
    tapping_intervals = load_tapping_data(file_path)

    # Compute tempo change percent in music cont
    tempo_change_cont_music = (np.mean((tapping_intervals[1])[-10:]) - np.mean((tapping_intervals[1])[:10]))*100/(60/bpms[song_id])

    # Compute tempo change percent in music cont
    tempo_change_cont_no_music = (np.mean((tapping_intervals[2])[-10:]) - np.mean((tapping_intervals[2])[:10]))*100/(60/bpms[song_id])

    # Compute tempo change percent in music cont
    tempo_change_total= (np.mean((tapping_intervals[2])[-10:]) - np.mean((tapping_intervals[1])[:10]))*100/(60/bpms[song_id])

    print(f"tempo_change_cont_music : {song_id} : {tempo_change_cont_music}")
    print(f"tempo_change_cont_no_music : {song_id} : {tempo_change_cont_no_music}")
    print(f"tempo_change_total : {song_id} : {tempo_change_total}")


tempo_change_cont_music : A : 1.118333333333041
tempo_change_cont_no_music : A : 4.026000000000008
tempo_change_total : A : 7.604666666666427
tempo_change_cont_music : B : -2.6116250000000285
tempo_change_cont_no_music : B : 2.3204999999999143
tempo_change_total : B : 8.625374999999998
tempo_change_cont_music : C : -1.871166666666521
tempo_change_cont_no_music : C : -0.6267500000000923
tempo_change_total : C : 0.8447500000001288
